In [ ]:
# !pip install selenium
# !pip install selenium chromedriver-autoinstaller
# !pip install webdriver-manager
# !pip install undetected-chromedriver
# !pip install scikit-image
#!pip install user_agent
#!pip install pyautogui

In [2]:
# !apt-get update
# !apt-get install -y chromium-browser
# !apt-get install chromium-chromedriver

In [1]:
import pandas as pd
from PIL import Image
import os
import random
import shutil
import re
import requests
import time
from time import sleep
import pyautogui
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [2]:
# Get the current working directory
current_dir = os.getcwd()
# Define download directory within the Midjourney folder
download_dir = os.path.join(current_dir, 'Midjourney')
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
    
# Define Chrome options and experimental preferences
options = uc.ChromeOptions()

options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument("--disable-popup-blocking")
options.add_argument(f"user-agent={user_agent.generate_user_agent()}")
prefs = {
    "profile.default_content_settings.popups": 0,
    "profile.content_settings.pattern_pairs.*.multiple-automatic-downloads": 1,
    "download.prompt_for_download": False,
    "download.default_directory": download_dir
}
options.add_experimental_option("prefs",prefs)
# Initialize undetected Chrome driver with the configured options
driver = uc.Chrome(options=options)

driver.get("https://discord.com/login")

In [3]:
driver.maximize_window();
driver.get("https://discord.com/channels/1155301271088152709/1155315618116022322")
# Open new tabs in the existing window with Bing Image Search for creating images
driver.execute_script("window.open('https://discord.com/channels/1155301271088152709/1238313223342653500', '_blank');")
driver.execute_script("window.open('https://discord.com/channels/1155301271088152709/1238313259279585362', '_blank');")
time.sleep(5)

In [4]:
# Read prompts from prompts.txt file
with open('prompts.txt', 'r') as file:
    prompts = file.readlines()
    
# Get screen dimensions for random movement within the screen
screen_width, screen_height = pyautogui.size()

# Randomize mouse movement within the screen
x = random.randint(0, screen_width)
y = random.randint(0, screen_height)

# Array to store the latest image source URLs for each tab
latest_image_urls = ["", "", ""]  # Initialize with empty strings for each tab
tweet_number = 1
# Loop through each prompt and generate images
for prompt_number, prompt_text in enumerate(prompts, start=1):
    # Loop to open new tabs and enter prompts
    for tab_index in range(len(driver.window_handles)):
        driver.switch_to.window(driver.window_handles[tab_index])
        
        pyautogui.moveTo(x, y, duration=random.uniform(0.5, 1))

        # Find the div element using its data-slate-node attribute
        div_element = driver.find_element(By.CSS_SELECTOR, 'div[data-slate-node="element"]')

        # Click on the div to focus on it
        ActionChains(driver).click(div_element).perform()

        # Wait for a second to ensure the cursor is focused
        time.sleep(random.uniform(1, 2))

        # Type the prompt text based on the prompt number
        ActionChains(driver).send_keys("/imagine ").perform()
        time.sleep(random.uniform(0.4, 0.6))
        ActionChains(driver).send_keys(prompts[(tweet_number + tab_index - 1) % len(prompts)].strip()).perform()

        # Press Enter to submit the command and prompt
        ActionChains(driver).send_keys(Keys.RETURN).perform()

    # Loop to get image source URLs and download images
    for tab_index in range(len(driver.window_handles)):
        driver.switch_to.window(driver.window_handles[tab_index])
        
        if(tab_index == 0):
            time.sleep(random.uniform(50, 70))

        pyautogui.moveTo(x, y, duration=random.uniform(0.5, 1))

        # Set download preferences
        prefs = {
            "profile.default_content_settings.popups": 0,
            "profile.content_settings.pattern_pairs.*.multiple-automatic-downloads": 1,
            "download.prompt_for_download": False,
            "download.default_directory": download_dir  # Use the specified download directory
        }
        options.add_experimental_option("prefs", prefs)

        # Get the latest div element
        latest_div_element = driver.find_elements(By.CSS_SELECTOR, 'div.loadingOverlay__35a04')[-1]

        # Extract the image source URL from the img tag within the div element
        image_src = latest_div_element.find_element(By.TAG_NAME, 'img').get_attribute('src')

        # Check if the image source URL is different from the latest for this tab
        if image_src != latest_image_urls[tab_index]:
            # Download the image using requests
            response = requests.get(image_src)
            if response.status_code == 200:
                # Save the image to a file with the Tweet number
                image_name = f"Tweet_{tweet_number}.png"
                
                with open(os.path.join(download_dir, image_name), "wb") as f:
                    f.write(response.content)
                    print(f"Image {image_name} saved successfully")
                    # Update the latest image source URL for this tab
                    latest_image_urls[tab_index] = image_src
            else:
                print(f"Failed to download the image for prompt {tweet_number}")
        else:
            print(f"Image for prompt {tweet_number} not created (same as previous)")
        prompt_number+=1
        tweet_number+= 1


Image Tweet_1.png saved successfully
Image Tweet_2.png saved successfully
Image Tweet_3.png saved successfully
Image Tweet_4.png saved successfully
Image Tweet_5.png saved successfully
Image Tweet_6.png saved successfully
Image Tweet_7.png saved successfully
Image Tweet_8.png saved successfully
Image for prompt 9 not created (same as previous)


: 

In [5]:
import os
from PIL import Image

# Define the path to the Midjourney folder
midjourney_dir = "Midjourney"  # Replace with the actual path to your Midjourney folder

# Define the path to the FMid folder
fmid_dir = "FMid"  # Replace with the actual path to your FMid folder
if not os.path.exists(fmid_dir):
    os.makedirs(fmid_dir)

# Loop through all files in the Midjourney folder
for filename in os.listdir(midjourney_dir):
    if filename.endswith(".png") or filename.endswith(".jpg"):  # Assuming images are PNG or JPG format
        # Open the image file
        image_path = os.path.join(midjourney_dir, filename)
        image = Image.open(image_path)

        # Get the dimensions of the image
        width, height = image.size

        # Calculate the dimensions for the top-left quarter (50% length, 50% breadth)
        quarter_width = width // 2
        quarter_height = height // 2

        # Crop the top-left quarter
        quarter_image = image.crop((0, 0, quarter_width, quarter_height))

        # Resize the cropped quarter to four times its original size
        resized_quarter = quarter_image.resize((quarter_width * 2, quarter_height * 2))

        # Define the new filename with the "Tweet_" prefix and appropriate extension
        new_filename = f"{filename.split('.')[0]}.png"

        # Save the resized quarter image in the FMid folder
        resized_quarter.save(os.path.join(fmid_dir, new_filename))


In [8]:
# Close the web driver
driver.quit()